<a href="https://colab.research.google.com/github/PatriciaReghine/Card-pio-IA/blob/main/card%C3%A1pio_para_as_empresas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai

In [10]:
from google.colab import userdata
api_key = userdata.get('APIKEY')

In [11]:
# --- Parte 1: Nessa parte sera gerado Embeddings para o Cardápio ---

model_embedding = "models/embedding-001"

# Dados do cardápio
cardapio = {
    "Dia da semana": ["Segunda", "Terça", "Quarta", "Quinta", "Sexta",
                      "Segunda: opção 2", "Terça: opção 2", "Quarta: opção 2",
                      "Quinta: opção 2", "Sexta: opção 2", "Opção disponível todos os dias"],
    "Proteína": ["Carne de boi", "Frango", "Carne de Boi", "Frango", "Peixe",
                 "Frango", "Carne de porco", "Frango", "Carne de Boi", "Carne de Boi", "Ovo"],
}
df_cardapio = pd.DataFrame(cardapio)

# Gerar embeddings para cada item do cardápio
df_cardapio["Embeddings"] = df_cardapio.apply(
    lambda row: genai.embed_content(
        model=model_embedding,
        content=row["Proteína"],
        task_type="RETRIEVAL_DOCUMENT"
    )["embedding"], axis=1
)


In [12]:
# --- Parte 2: Aqui serão utlizadas funções para registro de pedidos e recomendações ---

LIMITE_PEDIDOS = 100
total_pedidos = 0
pedidos_proteina = {}

def exibir_cardapio():
    print("Cardápio da Semana:")
    print(df_cardapio[["Dia da semana", "Proteína"]])

def registrar_pedido():
    global total_pedidos
    if total_pedidos >= LIMITE_PEDIDOS:
        print("Limite de pedidos atingido!")
        return

    exibir_cardapio()

    while True:
        opcao = input("Digite o número da sua opção (correspondente à linha da tabela): ")
        dia_semana = input("Digite o dia da semana: ")
        try:
            opcao = int(opcao)
            proteina = df_cardapio.loc[opcao, "Proteína"]

            # Registrar o pedido
            if dia_semana not in pedidos_proteina:
                pedidos_proteina[dia_semana] = {}
            if proteina not in pedidos_proteina[dia_semana]:
                pedidos_proteina[dia_semana][proteina] = 0
            pedidos_proteina[dia_semana][proteina] += 1
            total_pedidos += 1
            print(f"Pedido registrado: {proteina} no {dia_semana}")

            # Recomendar um prato similar
            recomendacao = recomendar_prato(proteina)
            if recomendacao:
                print(f"Você também pode gostar de: {recomendacao}")

            break
        except (ValueError, KeyError):
            print("Opção ou dia da semana inválido. Tente novamente.")

def recomendar_prato(proteina_escolhida):
    """Recomenda um prato similar à proteína escolhida."""

    embedding_escolhida = df_cardapio[df_cardapio["Proteína"] == proteina_escolhida]["Embeddings"].iloc[0]

    similaridades = df_cardapio["Embeddings"].apply(
        lambda embedding: np.dot(embedding, embedding_escolhida)
    )

    # Encontrar o prato mais similar
    df_similar = df_cardapio.copy()
    df_similar["Similaridade"] = similaridades
    df_similar = df_similar[df_similar["Proteína"] != proteina_escolhida]
    prato_similar = df_similar.sort_values(by="Similaridade", ascending=False).iloc[0]["Proteína"]

    return prato_similar


In [14]:
# --- Parte 3: Loop Principal ---

while True:
    comando = input("Digite um comando ('registrar', 'pedidos', 'sair'): ")
    if comando == 'registrar':
        registrar_pedido()
    elif comando == 'pedidos':
        print("\nPedidos de Proteína:")
        for dia, proteinas in pedidos_proteina.items():
            for proteina, quantidade in proteinas.items():
                if quantidade > 0:
                    print(f"{dia}: {quantidade} pedido(s) de {proteina}")
    elif comando == 'sair':
        break
    else:
        print("Comando inválido.")

Digite um comando ('registrar', 'pedidos', 'sair'): registrar
Cardápio da Semana:
                     Dia da semana        Proteína
0                          Segunda    Carne de boi
1                            Terça          Frango
2                           Quarta    Carne de Boi
3                           Quinta          Frango
4                            Sexta           Peixe
5                 Segunda: opção 2          Frango
6                   Terça: opção 2  Carne de porco
7                  Quarta: opção 2          Frango
8                  Quinta: opção 2    Carne de Boi
9                   Sexta: opção 2    Carne de Boi
10  Opção disponível todos os dias             Ovo
Digite o número da sua opção (correspondente à linha da tabela): 9
Digite o dia da semana: Sexta: opção 2
Pedido registrado: Carne de Boi no Sexta: opção 2
Você também pode gostar de: Carne de boi
Digite um comando ('registrar', 'pedidos', 'sair'): sair
